In [1]:
%cd ..

/kaggle/working


In [18]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../generate_datasets/001_baseline"):
    cfg = compose(config_name="config.yaml")
    print(OmegaConf.to_yaml(cfg))

debug: false
seed: 7
dir:
  data_dir: /kaggle/working/input/atmaCup16_Dataset
  output_dir: /kaggle/working/output
  exp_dir: /kaggle/working/output/exp
  cand_unsupervised_dir: /kaggle/working/output/cand_unsupervised
  cand_supervised_dir: /kaggle/working/output/cand_supervised
  datasets_dir: /kaggle/working/output/datasets
exp:
  fold_path: /kaggle/working/output/datasets/make_cv/base/train_fold.parquet
  candidate_info_list:
  - name: transition_prob/base
    max_num_candidates: 100
    dir: /kaggle/working/output/cand_unsupervised/transition_prob/base
    features:
    - name: yad2yad_feature
  - name: ranking_location/sml_cd
    max_num_candidates: 50
    dir: /kaggle/working/output/cand_unsupervised/ranking_location/sml_cd
    features:
    - name: yad_feature
  - name: ranking_location/lrg_cd
    max_num_candidates: 50
    dir: /kaggle/working/output/cand_unsupervised/ranking_location/lrg_cd
    features:
    - name: yad_feature



In [3]:
import os
import sys
from pathlib import Path

import hydra
import polars as pl
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf

import utils
from utils.load import load_label_data, load_log_data, load_session_data
from utils.metrics import calculate_metrics

In [37]:
mode = "train"

In [76]:
def load_and_union_candidates(cfg, mode: str):
    # logデータのsession中のyad_noを候補に加える
    log_df = load_log_data(Path(cfg.dir.data_dir), mode)
    df = log_df.group_by("session_id").agg(pl.col("yad_no").alias("candidates"))
    dfs = [df]
    for candidate_info in cfg.exp.candidate_info_list:
        df = pl.read_parquet(Path(candidate_info["dir"]) / f"{mode}_candidate.parquet")
        df = df.with_columns(
            pl.col("candidates")
            .list.head(candidate_info["max_num_candidates"])
            .alias("candidates")
        ).filter(pl.col("candidates").list.len() > 0)

        dfs.append(df)
    df = pl.concat(dfs)
    df = (
        df.group_by("session_id")
        .agg(pl.col("candidates").flatten())
        .with_columns(pl.col("candidates").list.unique())
    ).select(["session_id", "candidates"])

    # リストを展開
    candidate_df = df.explode("candidates")

    # セッション最後のyad_noを除外
    last_df = (
        load_log_data(Path(cfg.dir.data_dir), mode)
        .group_by("session_id")
        .agg(pl.col("yad_no").last().alias("candidates"))
        .with_columns(pl.lit(True).alias("last"))
        .sort(by="session_id")
    )
    candidate_df = (
        candidate_df.join(last_df, on=["session_id", "candidates"], how="left")
        .filter(pl.col("last").is_null())
        .drop("last")
    )
    return candidate_df

In [84]:
def convert_to_32bit(df):
    df = df.with_columns(
        [pl.col(col).cast(pl.Int32) for col in df.columns if df[col].dtype == pl.Int64]
    ).with_columns(
        [
            pl.col(col).cast(pl.Float32)
            for col in df.columns
            if df[col].dtype == pl.Float64
        ]
    )
    return df

In [ ]:
candidate_df = load_and_union_candidates(cfg, mode)

In [85]:
candidate_df = convert_to_32bit(candidate_df)
candidate_df.head()

session_id,candidates
str,i32
"""f34454bc86fa15…",38
"""f34454bc86fa15…",150
"""f34454bc86fa15…",488
"""f34454bc86fa15…",519
"""f34454bc86fa15…",780


In [ ]:
def concat_session_feature(cfg, mode: str, candidate_df: pl.DataFrame):
    """
    session_id, seq_no, yad_no に yado.csv を結合して集約し、セッションに関する特徴量を作成する
    """
    pass

In [ ]:
def concat_candidate_feature(cfg, mode: str, candidate_df: pl.DataFrame):
    """
    candidateの特徴量を抽出する
    """
    pass

In [ ]:
def concat_session_candidate_feature(cfg, mode: str, candidate_df: pl.DataFrame):
    """
    session中の特徴とcandidateの関係性を特徴量として抽出する
    例：session中におけるcandidateの出現回数(割合)、candidateと同一地域のものを見た回数(割合)
    """
    pass

In [ ]:
def concat_label_fold(cfg, mode: str, candidate_df):
    pass